lunana  
last update 2022 06 19

# Notebook  
* Foursquare EDA+sentence bert 実況  
https://www.kaggle.com/code/lunapandachan/foursquare-eda-sentence-bert  
* Foursquare s-bert labo note  
https://www.kaggle.com/code/lunapandachan/foursquare-s-bert-labo-note

* [**V0 Pretrain**](#V0-Pretrain)
      
* [**V1 Train**](#V1-Train)

* [**V2 Train fold<2**](#V2-Train-fold<2)

* [**V3 Train fold<4**](#V3-Train-fold<4)

* [**V4 Train fold<8**](#V4-Train-fold<8)

* [**V5 Train fold<16**](#V5-Train-fold<16)

* [**V6 Train fold<32**](#V6-Train-fold<32)

# Define

## switchis

In [ ]:
IS_V0=False
IS_V1=True
IS_V2=True
IS_V3=False
IS_V4=False
IS_V5=False
IS_V6=False
IS_TEST=False

## global values

In [ ]:
input_path='../input/foursquare-location-matching'
output_path='.'
ds_path='../input/foursquare-sbert-128fold-train-ds'

## CFG

In [ ]:
class CFG:
    pairs='pairs.csv'
    train='train.csv'
    sup_train='sup_train.csv'
    eval_name='eval.csv'    
    #    s_size=5000
    device='cuda'
#    n_sample=40000
    seed=42
    batch_size=24
    epochs=1
#    epochs=5
    repe1=10
    repe2=5
    repe3=5
    repe4=3
    repe5=2
    repe6=1
    repe7=1
    model_name='paraphrase-multilingual-MiniLM-L12-v2'
    pickle_name_v0='/paraphrase-multilingual-MiniLM-L12-v2-0.pkl'
    pickle_name_v1='/paraphrase-multilingual-MiniLM-L12-v2-1.pkl'
    pickle_name_v2='/paraphrase-multilingual-MiniLM-L12-v2-2.pkl'
    pickle_name_v3='/paraphrase-multilingual-MiniLM-L12-v2-3.pkl'
    pickle_name_v4='/paraphrase-multilingual-MiniLM-L12-v2-4.pkl'
    pickle_name_v5='/paraphrase-multilingual-MiniLM-L12-v2-5.pkl'
    pickle_name_v6='/paraphrase-multilingual-MiniLM-L12-v2-6.pkl'
    emb_name='/emb.pkl'

# Initialize

In [ ]:
import os
import pandas as pd
import numpy as np
import pickle
import gc
from IPython.display import Image
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import seaborn as sns
import matplotlib.cm as cm
import matplotlib.pylab as plt
from tqdm import tqdm
import torch

## union find from lunana library

In [ ]:
import sys
sys.path.append("../input/lunana-library")
from union_find import UnionFind

# V0 Pretrain

In [ ]:
def make_text(df):
    df['text'] = df['latitude'].astype(str)+ '[SEP]'+df['longitude'].astype(str)+ '[SEP]'+df['name'] + '[SEP]' + df['address'] + '[SEP]' + df['city'] + '[SEP]' + df['state'] + '[SEP]' + df['country'] + '[SEP]' + df['url'] + '[SEP]' + df['categories'] 
    return df

In [ ]:
def make_train_supplement():
    df = pd.read_csv(input_path+'/'+CFG.train)
    df = make_text(df.fillna("").reset_index())
    return df

In [ ]:
def get_train_supplement( fn ):
    df = pd.read_csv(fn)
    return df

In [ ]:
if IS_V0:
    df_train=make_train_supplement()

# Dataset

## K-fold

### k-fold+shuffle

In [ ]:
from sklearn.model_selection import KFold
if IS_V0:
    df_train_g=df_train.groupby('point_of_interest').size()
    df_train_g_small=df_train_g.sort_values(ascending=False).copy()
    df_train_idx=df_train_g_small.keys()

    kf = KFold(n_splits=128,random_state=0, shuffle=True) 
    kfr=kf.split(df_train_idx)
    df=pd.DataFrame(df_train_idx)

    for i, (tr_idx, va_idx) in enumerate(kf.split(df_train_idx)):
        df.loc[va_idx,'fold']=i
    df=df.astype({'fold': 'int'})    
    df_train=df_train.merge(df,on=['point_of_interest'],how='left')
    df_train1=df_train[['index','id','text','point_of_interest','fold']].copy()
    df_train=df_train1
    df_train[['index','id','text','point_of_interest','fold']].to_csv(output_path+'/'+CFG.sup_train, index=False)
else:
    df_train=get_train_supplement(ds_path+'/'+CFG.sup_train)


# Model

In [ ]:
!pip install -U sentence-transformers
import sentence_transformers

In [ ]:
if IS_V0:
    model = sentence_transformers.SentenceTransformer(CFG.model_name,device=CFG.device)
    with open(output_path+CFG.pickle_name_v0, 'wb') as f:
        pickle.dump(model, f)    
else:
    with open(ds_path+CFG.pickle_name_v0, 'rb') as f:
        model=pickle.load( f )

# eval pretrain

## df_eval

In [ ]:
if IS_V0:
    df_eval=df_train.query('fold>100').copy()
    df_eval_g=df_eval.groupby('point_of_interest').size()
    df_eval_g_small=df_eval_g.sort_values(ascending=False)[:12].copy()
    df_eval_idx=df_eval_g_small.keys()
    df_eval=df_eval[df_eval['point_of_interest'].isin(df_eval_idx)].copy()

    del df_eval_g,df_eval_g_small,df_eval_idx

    df_eval=df_eval.drop('index',axis=1).reset_index()
    df_eval=df_eval.drop('index',axis=1).reset_index()
    df_eval.to_csv(output_path+'/'+CFG.eval_name,index=False)
    
else:
    df_eval=pd.read_csv(ds_path+'/'+CFG.eval_name)

## fig

In [ ]:
def make_fig(df,embed,dict_poi,save_fn='fig.png'):
    tsne = TSNE(
      n_components=2, random_state=0
    )
    # SCDVの次元削減をする
    tsne2=tsne.fit_transform(embed)
    colors = [cm.jet(dict_poi[i]/(len(dict_poi)-1)) for i in df['point_of_interest']]
    fig=plt.figure(figsize=(10, 10))
    plt.scatter(tsne2[:, 0], tsne2[:, 1], c=colors)
    fig.savefig(save_fn)

In [ ]:
if IS_V0:
    sentences = df_eval['text'].tolist()
    embeddings = model.encode(sentences)
    df_eval_g=df_eval.groupby('point_of_interest').size()
    df_eval_idx=df_eval_g.keys()
    dict_eval = {df_eval_idx[i]: i for i in range(len(df_eval_idx))}
    make_fig(df_eval,embeddings,dict_eval,save_fn=output_path+'/fig1pre.png')    
else:
    obj=plt.imread(ds_path+'/fig1pre.png')
    plt.figure(figsize=(10,10))
    plt.imshow(obj)

# V1 Train

## dataset and train

In [ ]:
from sentence_transformers import SentenceTransformer, InputExample,losses
from torch.utils.data import DataLoader

In [ ]:
torch.manual_seed(CFG.seed)

In [ ]:
model = sentence_transformers.SentenceTransformer(CFG.model_name,device=CFG.device)

In [ ]:
if IS_V1:
    train_ds=df_train.query('fold<1').copy()
    train_ds=train_ds.drop('index',axis=1).reset_index()
    train_ds=train_ds.drop('index',axis=1).reset_index()
    train_ds_g=train_ds.groupby('point_of_interest').groups
    
    sentences = train_ds['text'].tolist()


In [ ]:
def train(train_ds,model):

    embeddings=model.encode(sentences)

    emb=np.array(embeddings)
    del embeddings
    gc.collect()
    emb_n=np.array([emb[i]/np.linalg.norm(emb[i], ord=2) for i in range(len(emb))])
    del emb
    gc.collect()

    list_ds=[]
    for idx in tqdm(range(len(emb_n))):
        b_emb=emb_n[idx]
        c_s=np.dot(emb_n,b_emb.T).T
        poi=train_ds.loc[idx,'point_of_interest']
        group_poi=train_ds_g[poi]
        c_s[group_poi]=0
        
        if len(group_poi)>1:
            over_c_s=np.argmax(c_s)
            for p_idx in group_poi:
                list_ds.append([idx,p_idx,over_c_s])
        else:
            over_c_s=np.where(c_s>0.95)
            for p_idx in group_poi:
                for n_idx in over_c_s[0]:
                    list_ds.append([idx,p_idx,n_idx])
                    
    l_df=len(list_ds)
    print(l_df)
    train_examples=[]
    for idx in tqdm(range(l_df)):
        train_examples.append(InputExample(texts=[train_ds.loc[list_ds[idx][0],'text'],train_ds.loc[list_ds[idx][1],'text'],train_ds.loc[list_ds[idx][2],'text']]))
    train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=CFG.batch_size)
        
    train_loss = losses.TripletLoss(model)

    model.fit(train_objectives=[(train_dataloader, train_loss)], epochs=CFG.epochs, warmup_steps=100)
    return model  

## train

In [ ]:
if IS_V1:
    for i in range(CFG.repe1):
        model=train(train_ds,model)
    with open(output_path+CFG.pickle_name_v1, 'wb') as f:
        pickle.dump(model, f)
else:
    with open(ds_path+CFG.pickle_name_v1, 'rb') as f:
        model=pickle.load( f )

## post train eval fig

In [ ]:
if IS_V1:
    sentences = df_eval['text'].tolist()
    embeddings = model.encode(sentences)
    df_eval_g=df_eval.groupby('point_of_interest').size()
    df_eval_idx=df_eval_g.keys()
    dict_eval = {df_eval_idx[i]: i for i in range(len(df_eval_idx))}
    make_fig(df_eval,embeddings,dict_eval,save_fn=output_path+'/fig1post.png')    
else:
    obj=plt.imread(ds_path+'/fig1post.png')
    plt.figure(figsize=(10,10))
    plt.imshow(obj)

# V2 Train fold<2

## dataset

In [ ]:
if IS_V2:
    train_examples = []

    train_ds=df_train.query('fold<2').copy()
    train_ds=train_ds.drop('index',axis=1).reset_index()
    train_ds=train_ds.drop('index',axis=1).reset_index()
    train_ds_g=train_ds.groupby('point_of_interest').groups
    sentences = train_ds['text'].tolist()
    
    if not IS_V1:
        with open(ds_path+CFG.pickle_name_v1, 'rb') as f:
            model=pickle.load( f )

## train

In [ ]:
if IS_V2:
    for i in range(CFG.repe2):
        model=train(train_ds,model)
    with open(output_path+CFG.pickle_name_v2, 'wb') as f:
        pickle.dump(model, f)
else:
    with open(ds_path+CFG.pickle_name_v2, 'rb') as f:
        model=pickle.load( f )

## post train eval fig

In [ ]:
if IS_V2:
    sentences = df_eval['text'].tolist()
    embeddings = model.encode(sentences)
    df_eval_g=df_eval.groupby('point_of_interest').size()
    df_eval_idx=df_eval_g.keys()
    dict_eval = {df_eval_idx[i]: i for i in range(len(df_eval_idx))}
    make_fig(df_eval,embeddings,dict_eval,save_fn=output_path+'/fig2post.png')    
else:
    obj=plt.imread(ds_path+'/fig2post.png')
    plt.figure(figsize=(10,10))
    plt.imshow(obj)

# V3 Train fold<4

## dataset

In [ ]:
if IS_V3:
    train_ds=df_train.query('fold<4').copy()
    train_ds=train_ds.drop('index',axis=1).reset_index()
    train_ds=train_ds.drop('index',axis=1).reset_index()
    train_ds_g=train_ds.groupby('point_of_interest').groups
    sentences = train_ds['text'].tolist()
    
    if not IS_V2:
        with open(ds_path+CFG.pickle_name_v2, 'rb') as f:
            model=pickle.load( f )

## train

In [ ]:
if IS_V3:
    for i in range(CFG.repe3):
        model=train(train_ds,model)
    with open(output_path+CFG.pickle_name_v3, 'wb') as f:
        pickle.dump(model, f)
else:
    with open(ds_path+CFG.pickle_name_v3, 'rb') as f:
        model=pickle.load( f )

## fig

In [ ]:
if IS_V3:
    sentences = df_eval['text'].tolist()
    embeddings = model.encode(sentences)
    df_eval_g=df_eval.groupby('point_of_interest').size()
    df_eval_idx=df_eval_g.keys()
    dict_eval = {df_eval_idx[i]: i for i in range(len(df_eval_idx))}
    make_fig(df_eval,embeddings,dict_eval,save_fn=output_path+'/fig3post.png')    
else:
    obj=plt.imread(ds_path+'/fig3post.png')
    plt.figure(figsize=(10,10))
    plt.imshow(obj)

# V4 Train fold<8

## dataset

In [ ]:
if IS_V4:
    train_ds=df_train.query('fold<8').copy()
    train_ds=train_ds.drop('index',axis=1).reset_index()
    train_ds=train_ds.drop('index',axis=1).reset_index()
    train_ds_g=train_ds.groupby('point_of_interest').groups
    sentences = train_ds['text'].tolist()
    
    if not IS_V3:
        with open(ds_path+CFG.pickle_name_v3, 'rb') as f:
            model=pickle.load( f )

## train

In [ ]:
if IS_V4:
    for i in range(CFG.repe4):
        model=train(train_ds,model)
    with open(output_path+CFG.pickle_name_v4, 'wb') as f:
        pickle.dump(model, f)
else:
    with open(ds_path+CFG.pickle_name_v4, 'rb') as f:
        model=pickle.load( f )

## fig

In [ ]:
if IS_V4:
    sentences = df_eval['text'].tolist()
    embeddings = model.encode(sentences)
    df_eval_g=df_eval.groupby('point_of_interest').size()
    df_eval_idx=df_eval_g.keys()
    dict_eval = {df_eval_idx[i]: i for i in range(len(df_eval_idx))}
    make_fig(df_eval,embeddings,dict_eval,save_fn=output_path+'/fig4post.png')    
else:
    obj=plt.imread(ds_path+'/fig4post.png')
    plt.figure(figsize=(10,10))
    plt.imshow(obj)

# Test

In [ ]:
test_ds=df_train.query('fold>120').copy()
test_ds=test_ds.drop('index',axis=1).reset_index()
test_ds=test_ds.drop('index',axis=1).reset_index()

In [ ]:
def test(test_ds,model,th):
    test_ds_g=test_ds.groupby('point_of_interest').groups
    sentences = test_ds['text'].tolist()
    embeddings=model.encode(sentences)

    emb=np.array(embeddings)
    del embeddings
    gc.collect()
    emb_n=np.array([emb[i]/np.linalg.norm(emb[i], ord=2) for i in range(len(emb))])
    del emb
    gc.collect()
    
    uf =UnionFind(len(test_ds))
    for idx in tqdm(range(len(emb_n))):
        b_emb=emb_n[idx]
        c_s=np.dot(emb_n,b_emb.T).T
        over_c_s=np.where(c_s>th)
        for p_idx in over_c_s[0]:
            uf.union(idx,p_idx)
    total_or=0
    total_and=0
    total_ideal=0
    for idx in tqdm(range(len(emb_n))):
        poi=test_ds.loc[idx,'point_of_interest']
        group_poi=test_ds_g[poi]
        total_or+=len(group_poi)
        total_ideal+=len(group_poi)
        p_list=uf.members(idx)
        for p_idx in p_list:
            if p_idx in group_poi:
                total_and+=1
            else:
                total_or+=1
    return total_ideal,total_and,total_or,total_and/total_or

In [ ]:
if IS_TEST:
    with open(ds_path+CFG.pickle_name_v4, 'rb') as f:
        model=pickle.load( f )
    res_i,res_a,res_o,res=test(test_ds,model,0.98)
    print(res_i,res_a,res_o,res)

# V5 Train fold<16

## dataset

In [ ]:
if IS_V5:
    train_ds=df_train.query('fold<16').copy()
    train_ds=train_ds.drop('index',axis=1).reset_index()
    train_ds=train_ds.drop('index',axis=1).reset_index()
    train_ds_g=train_ds.groupby('point_of_interest').groups
    sentences = train_ds['text'].tolist()
    
    if not IS_V4:
        with open(ds_path+CFG.pickle_name_v4, 'rb') as f:
            model=pickle.load( f )

## train

In [ ]:
if IS_V5:
    for i in range(CFG.repe5):
        model=train(train_ds,model)
    with open(output_path+CFG.pickle_name_v5, 'wb') as f:
        pickle.dump(model, f)
else:
    with open(ds_path+CFG.pickle_name_v5, 'rb') as f:
        model=pickle.load( f )

## fig

In [ ]:
if IS_V5:
    sentences = df_eval['text'].tolist()
    embeddings = model.encode(sentences)
    df_eval_g=df_eval.groupby('point_of_interest').size()
    df_eval_idx=df_eval_g.keys()
    dict_eval = {df_eval_idx[i]: i for i in range(len(df_eval_idx))}
    make_fig(df_eval,embeddings,dict_eval,save_fn=output_path+'/fig5post.png')    
else:
    obj=plt.imread(ds_path+'/fig5post.png')
    plt.figure(figsize=(10,10))
    plt.imshow(obj)

# V6 Train fold<32

## dataset

In [ ]:
if IS_V6:
    train_ds=df_train.query('fold<32').copy()
    train_ds=train_ds.drop('index',axis=1).reset_index()
    train_ds=train_ds.drop('index',axis=1).reset_index()
    train_ds_g=train_ds.groupby('point_of_interest').groups
    sentences = train_ds['text'].tolist()
    
    if not IS_V5:
        with open(ds_path+CFG.pickle_name_v5, 'rb') as f:
            model=pickle.load( f )

## train

In [ ]:
if IS_V6:
    for i in range(CFG.repe6):
        model=train(train_ds,model)
    with open(output_path+CFG.pickle_name_v6, 'wb') as f:
        pickle.dump(model, f)
else:
    with open(ds_path+CFG.pickle_name_v6, 'rb') as f:
        model=pickle.load( f )

## fig

In [ ]:
if IS_V6:
    sentences = df_eval['text'].tolist()
    embeddings = model.encode(sentences)
    df_eval_g=df_eval.groupby('point_of_interest').size()
    df_eval_idx=df_eval_g.keys()
    dict_eval = {df_eval_idx[i]: i for i in range(len(df_eval_idx))}
    make_fig(df_eval,embeddings,dict_eval,save_fn=output_path+'/fig6post.png')    
else:
    obj=plt.imread(ds_path+'/fig6post.png')
    plt.figure(figsize=(10,10))
    plt.imshow(obj)